In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ShopByAttribute_Products.csv')
df

,ProductID,King,Queen,Bay,Ocean,Elevator,Total
0,XXXX,1,0,1,0,0,3
1,YYYY,1,0,0,1,0,3
2,ZZZZ,1,0,0,0,1,2
3,AAAA,0,1,1,0,0,3
4,BBBB,0,1,0,1,0,3
5,CCCC,0,1,0,0,1,2


In [3]:
Attributes = ['King', 'Queen', 'Bay', 'Ocean', 'Elevator']

In [4]:
def concat(a, b):
    return str(a) + str(b)

In [5]:
Products = {df.loc[j, 'ProductID']: list(df.loc[j, Attributes].values) for j in range (len(df)) }

In [6]:
for i,j in Products.items():
    vector = 0
    for v in range(len(j)):
        vector = concat(vector, j[v])
    Products[i] = vector 

In [7]:
Products

{'XXXX': '010100',
 'YYYY': '010010',
 'ZZZZ': '010001',
 'AAAA': '001100',
 'BBBB': '001010',
 'CCCC': '001001'}

In [8]:
ProductsInAttributeSpace = []
for p in Products.keys():
    n = df[df['ProductID'] == p]['Total'].values[0]
    for i in range(n):
        ProductsInAttributeSpace.append(Products[p])
        
promised = {0: ['0']}
prevFulfillment = {'0': ['0']}

In [9]:
ProductsInAttributeSpace

['010100',
 '010100',
 '010100',
 '010010',
 '010010',
 '010010',
 '010001',
 '010001',
 '001100',
 '001100',
 '001100',
 '001010',
 '001010',
 '001010',
 '001001',
 '001001']

In [10]:
def requestIntoAttributeSpace(attributes, request):
    ra = 0
    for a in attributes:
        if a in request:
            ra = concat(ra, 1)
        else:
            ra = concat(ra, 0)
    return ra
    

In [11]:
requestIntoAttributeSpace(attributes = Attributes, request = ["King", "Bay"])

'010100'

In [12]:
requestIntoAttributeSpace(attributes = Attributes, request = ["King"])

'010000'

In [13]:
requestIntoAttributeSpace(attributes = Attributes, request = ["Queen", "Bay"])

'001100'

In [14]:
requestIntoAttributeSpace(attributes = Attributes, request = ["Queen"])

'001000'

In [15]:
def getOrderedSet(a):
    r = set()
    for e in a:
        r.add(e)
    return r

In [16]:
import re

def encodeString(string):
    string = re.sub(r'<REQ-[0-9]+>', r'x', string)
    return string

def decodeString(stringAsList, encodedPortions, previousComposition, deltaLength):
    
    StEn = {}
    if deltaLength > 0:
        for i,j in encodedPortions.keys():
            ii = i + deltaLength
            jj = j + deltaLength
            StEn[(ii,jj)] = encodedPortions[(i,j)]
    else:
        StEn = encodedPortions
    
    i = 0
    j = 0
    for ii in range(len(stringAsList)):
        if stringAsList[ii] == 'x':
            stringAsList[ii] = previousComposition[i]
            i = i + 1
    
    decodedString =  ''.join(stringAsList)
    start = find(decodedString, ch = '<')
    end = find(decodedString, ch = '>')
    recordKeeping = dict()
    assert len(start) == len(end)
    if len(start)> 0:
        for i, j in zip(start, end):
            substr = decodedString[i:j+1]
            recordKeeping[(i,j)] = substr
    dif = set(StEn.items()) - set(recordKeeping.items())
    assert len(dif) == 0
    
    return decodedString

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

In [17]:
def evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request, prevFulfillment):
    
#     #Validation
#     for k in prevFulfillment.keys():
#         pfs = prevFulfillment[k]
#         paddedPfs = []
#         for pf in pfs:
#             if len(pf) < len(ProductsInAttributeSpace):
#                 pf = pf.rjust(len(ProductsInAttributeSpace), '0')
#             paddedPfs.append(pf)
            
#         prevFulfillment[k] = paddedPfs
    
    ra = requestIntoAttributeSpace(attributes = Attributes, request = request)
    
    satisfy = [int(int(ra,2) == (int(ra,2) & int(p,2))) for p in ProductsInAttributeSpace]
    
    if sum(satisfy) == 0:
        print ("Request can't be fulfilled")
        return promised, prevFulfillment
    
    else:
        howManySatisfied = []
        whichOnesStatisfied = [i for i in range(len(satisfy)) if satisfy[i] == 1 ]
        for i in whichOnesStatisfied:
            howManySatisfied.append(1 << (len(ProductsInAttributeSpace) - i - 1))
            
        howManySatisfied = ["{0:b}".format(i) for i in howManySatisfied]
        
        latestPromised = promised[max(promised.keys())]
        currentReqId = str(max(promised.keys())+1)
        
        temp = []
        potentialFulfillment = {}
        
        for lp in latestPromised:
            for hs in howManySatisfied:
                check = (int(lp,2) ^ int(hs,2))
                check = check &  int(hs,2)
                if check > 0:
                    CanBeFulfilledLike  = (int(lp,2) | int(hs,2))
                    CanBeFulfilledLike = "{0:b}".format(CanBeFulfilledLike).rjust(len(ProductsInAttributeSpace), '0')
                    temp.append(CanBeFulfilledLike)
                    
                    # Generating Specific Combinations to Fulfill the Requests
                    if CanBeFulfilledLike not in potentialFulfillment.keys():
                        potentialFulfillment[CanBeFulfilledLike] = []
                        
                    tempFulfillment = potentialFulfillment[CanBeFulfilledLike]
                    previousFulfillment = prevFulfillment[lp]
                    prdToReq = hs.rjust(len(ProductsInAttributeSpace), '0').find('1')
                    
                    for pf in previousFulfillment:
                        start = find(pf, ch = '<')
                        end = find(pf, ch = '>')
                        assert len(start) == len(end)
                        previousComposition = []
                        recordKeeping = dict()
                        if len(start)> 0:
                            for i, j in zip(start, end):
                                substr = pf[i:j+1]
                                recordKeeping[(i,j)] = substr
                                previousComposition.append(substr)
                                
                        encodedString = encodeString(pf)        
                        lb = len(encodedString)
                        encodedString = encodedString.rjust(len(ProductsInAttributeSpace), '0')
                        la = len(encodedString)
                        delta = la - lb
                        encodedStringAsList = list(encodedString)
                        encodedStringAsList[prdToReq] = 'y'
                        if len(start)> 0:
                            decodedString = decodeString(encodedStringAsList, recordKeeping, 
                                                         previousComposition, deltaLength=delta)
                        else:
                            decodedString = ''.join(encodedStringAsList)
                            
                        decodedString = decodedString.replace('y', '<REQ-{}>'.format(currentReqId), 1)
                        tempFulfillment.append(decodedString)
                        
                    tempFulfillment = list(set(tempFulfillment))
                    potentialFulfillment[CanBeFulfilledLike] = tempFulfillment
                    
                    
        if len(temp) > 0:
            print ("Congratulations Your Request Can be fulfilled")
            temp = list(getOrderedSet(temp))
            tempFulfillment = list(getOrderedSet(tempFulfillment))
            promised[max(promised.keys())+1] = temp
            
        else:
            
            print ("Sorry! Request can't be fulfilled")
            potentialFulfillment = prevFulfillment

    return promised, potentialFulfillment
    

In [18]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, 
                                            request=["Elevator"], prevFulfillment = prevFulfillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['0000000000000010',
 '0000000000000001',
 '0000001000000000',
 '0000000100000000']

In [19]:
fulFillment

{'0000001000000000': ['000000<REQ-1>000000000'],
 '0000000100000000': ['0000000<REQ-1>00000000'],
 '0000000000000010': ['00000000000000<REQ-1>0'],
 '0000000000000001': ['000000000000000<REQ-1>']}

In [20]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes,
                                            request=["Elevator"], prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['0000000000000011',
 '0000001000000010',
 '0000000100000010',
 '0000001100000000',
 '0000001000000001',
 '0000000100000001']

In [21]:
fulFillment

{'0000001000000010': ['000000<REQ-1>0000000<REQ-2>0',
  '000000<REQ-2>0000000<REQ-1>0'],
 '0000000100000010': ['0000000<REQ-2>000000<REQ-1>0',
  '0000000<REQ-1>000000<REQ-2>0'],
 '0000000000000011': ['00000000000000<REQ-1><REQ-2>',
  '00000000000000<REQ-2><REQ-1>'],
 '0000001000000001': ['000000<REQ-2>00000000<REQ-1>',
  '000000<REQ-1>00000000<REQ-2>'],
 '0000000100000001': ['0000000<REQ-1>0000000<REQ-2>',
  '0000000<REQ-2>0000000<REQ-1>'],
 '0000001100000000': ['000000<REQ-2><REQ-1>00000000',
  '000000<REQ-1><REQ-2>00000000']}

In [22]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["Elevator"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['0000001000000011',
 '0000001100000010',
 '0000000100000011',
 '0000001100000001']

In [23]:
fulFillment

{'0000001000000011': ['000000<REQ-3>0000000<REQ-2><REQ-1>',
  '000000<REQ-3>0000000<REQ-1><REQ-2>',
  '000000<REQ-1>0000000<REQ-3><REQ-2>',
  '000000<REQ-2>0000000<REQ-1><REQ-3>',
  '000000<REQ-2>0000000<REQ-3><REQ-1>',
  '000000<REQ-1>0000000<REQ-2><REQ-3>'],
 '0000000100000011': ['0000000<REQ-3>000000<REQ-1><REQ-2>',
  '0000000<REQ-2>000000<REQ-3><REQ-1>',
  '0000000<REQ-1>000000<REQ-2><REQ-3>',
  '0000000<REQ-3>000000<REQ-2><REQ-1>',
  '0000000<REQ-1>000000<REQ-3><REQ-2>',
  '0000000<REQ-2>000000<REQ-1><REQ-3>'],
 '0000001100000010': ['000000<REQ-2><REQ-1>000000<REQ-3>0',
  '000000<REQ-1><REQ-3>000000<REQ-2>0',
  '000000<REQ-3><REQ-2>000000<REQ-1>0',
  '000000<REQ-1><REQ-2>000000<REQ-3>0',
  '000000<REQ-2><REQ-3>000000<REQ-1>0',
  '000000<REQ-3><REQ-1>000000<REQ-2>0'],
 '0000001100000001': ['000000<REQ-3><REQ-1>0000000<REQ-2>',
  '000000<REQ-1><REQ-3>0000000<REQ-2>',
  '000000<REQ-1><REQ-2>0000000<REQ-3>',
  '000000<REQ-2><REQ-3>0000000<REQ-1>',
  '000000<REQ-3><REQ-2>0000000<REQ-1>

In [24]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["Elevator"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['0000001100000011']

In [25]:
fulFillment

{'0000001100000011': ['000000<REQ-4><REQ-2>000000<REQ-1><REQ-3>',
  '000000<REQ-3><REQ-2>000000<REQ-4><REQ-1>',
  '000000<REQ-4><REQ-1>000000<REQ-2><REQ-3>',
  '000000<REQ-3><REQ-1>000000<REQ-2><REQ-4>',
  '000000<REQ-2><REQ-4>000000<REQ-1><REQ-3>',
  '000000<REQ-1><REQ-2>000000<REQ-4><REQ-3>',
  '000000<REQ-3><REQ-4>000000<REQ-2><REQ-1>',
  '000000<REQ-4><REQ-3>000000<REQ-2><REQ-1>',
  '000000<REQ-2><REQ-4>000000<REQ-3><REQ-1>',
  '000000<REQ-1><REQ-4>000000<REQ-3><REQ-2>',
  '000000<REQ-4><REQ-2>000000<REQ-3><REQ-1>',
  '000000<REQ-1><REQ-4>000000<REQ-2><REQ-3>',
  '000000<REQ-2><REQ-1>000000<REQ-3><REQ-4>',
  '000000<REQ-1><REQ-3>000000<REQ-2><REQ-4>',
  '000000<REQ-2><REQ-3>000000<REQ-1><REQ-4>',
  '000000<REQ-3><REQ-2>000000<REQ-1><REQ-4>',
  '000000<REQ-4><REQ-3>000000<REQ-1><REQ-2>',
  '000000<REQ-2><REQ-3>000000<REQ-4><REQ-1>',
  '000000<REQ-2><REQ-1>000000<REQ-4><REQ-3>',
  '000000<REQ-3><REQ-4>000000<REQ-1><REQ-2>',
  '000000<REQ-3><REQ-1>000000<REQ-4><REQ-2>',
  '000000<REQ-

In [26]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["King", "Elevator"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Sorry! Request can't be fulfilled


['0000001100000011']

In [27]:
fulFillment

{'0000001100000011': ['000000<REQ-4><REQ-2>000000<REQ-1><REQ-3>',
  '000000<REQ-3><REQ-2>000000<REQ-4><REQ-1>',
  '000000<REQ-4><REQ-1>000000<REQ-2><REQ-3>',
  '000000<REQ-3><REQ-1>000000<REQ-2><REQ-4>',
  '000000<REQ-2><REQ-4>000000<REQ-1><REQ-3>',
  '000000<REQ-1><REQ-2>000000<REQ-4><REQ-3>',
  '000000<REQ-3><REQ-4>000000<REQ-2><REQ-1>',
  '000000<REQ-4><REQ-3>000000<REQ-2><REQ-1>',
  '000000<REQ-2><REQ-4>000000<REQ-3><REQ-1>',
  '000000<REQ-1><REQ-4>000000<REQ-3><REQ-2>',
  '000000<REQ-4><REQ-2>000000<REQ-3><REQ-1>',
  '000000<REQ-1><REQ-4>000000<REQ-2><REQ-3>',
  '000000<REQ-2><REQ-1>000000<REQ-3><REQ-4>',
  '000000<REQ-1><REQ-3>000000<REQ-2><REQ-4>',
  '000000<REQ-2><REQ-3>000000<REQ-1><REQ-4>',
  '000000<REQ-3><REQ-2>000000<REQ-1><REQ-4>',
  '000000<REQ-4><REQ-3>000000<REQ-1><REQ-2>',
  '000000<REQ-2><REQ-3>000000<REQ-4><REQ-1>',
  '000000<REQ-2><REQ-1>000000<REQ-4><REQ-3>',
  '000000<REQ-3><REQ-4>000000<REQ-1><REQ-2>',
  '000000<REQ-3><REQ-1>000000<REQ-4><REQ-2>',
  '000000<REQ-

In [28]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["King", "Bay"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['1000001100000011', '0010001100000011', '0100001100000011']

In [29]:
fulFillment

{'1000001100000011': ['<REQ-5>00000<REQ-2><REQ-3>000000<REQ-1><REQ-4>',
  '<REQ-5>00000<REQ-2><REQ-3>000000<REQ-4><REQ-1>',
  '<REQ-5>00000<REQ-1><REQ-4>000000<REQ-3><REQ-2>',
  '<REQ-5>00000<REQ-3><REQ-1>000000<REQ-2><REQ-4>',
  '<REQ-5>00000<REQ-1><REQ-3>000000<REQ-4><REQ-2>',
  '<REQ-5>00000<REQ-4><REQ-3>000000<REQ-2><REQ-1>',
  '<REQ-5>00000<REQ-1><REQ-2>000000<REQ-3><REQ-4>',
  '<REQ-5>00000<REQ-3><REQ-4>000000<REQ-2><REQ-1>',
  '<REQ-5>00000<REQ-3><REQ-4>000000<REQ-1><REQ-2>',
  '<REQ-5>00000<REQ-4><REQ-1>000000<REQ-2><REQ-3>',
  '<REQ-5>00000<REQ-2><REQ-4>000000<REQ-1><REQ-3>',
  '<REQ-5>00000<REQ-1><REQ-4>000000<REQ-2><REQ-3>',
  '<REQ-5>00000<REQ-4><REQ-3>000000<REQ-1><REQ-2>',
  '<REQ-5>00000<REQ-3><REQ-1>000000<REQ-4><REQ-2>',
  '<REQ-5>00000<REQ-1><REQ-3>000000<REQ-2><REQ-4>',
  '<REQ-5>00000<REQ-3><REQ-2>000000<REQ-4><REQ-1>',
  '<REQ-5>00000<REQ-4><REQ-2>000000<REQ-1><REQ-3>',
  '<REQ-5>00000<REQ-2><REQ-1>000000<REQ-3><REQ-4>',
  '<REQ-5>00000<REQ-1><REQ-2>000000<REQ-4><R

In [30]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["Queen", "Bay"],
                               prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['0010001101000011',
 '0100001110000011',
 '0100001100100011',
 '1000001101000011',
 '0010001100100011',
 '0010001110000011',
 '0100001101000011',
 '1000001110000011',
 '1000001100100011']

In [31]:
fulFillment

{'1000001110000011': ['<REQ-5>00000<REQ-4><REQ-1><REQ-6>00000<REQ-2><REQ-3>',
  '<REQ-5>00000<REQ-2><REQ-4><REQ-6>00000<REQ-1><REQ-3>',
  '<REQ-5>00000<REQ-2><REQ-4><REQ-6>00000<REQ-3><REQ-1>',
  '<REQ-5>00000<REQ-2><REQ-1><REQ-6>00000<REQ-4><REQ-3>',
  '<REQ-5>00000<REQ-2><REQ-1><REQ-6>00000<REQ-3><REQ-4>',
  '<REQ-5>00000<REQ-4><REQ-1><REQ-6>00000<REQ-3><REQ-2>',
  '<REQ-5>00000<REQ-3><REQ-1><REQ-6>00000<REQ-2><REQ-4>',
  '<REQ-5>00000<REQ-2><REQ-3><REQ-6>00000<REQ-4><REQ-1>',
  '<REQ-5>00000<REQ-1><REQ-3><REQ-6>00000<REQ-4><REQ-2>',
  '<REQ-5>00000<REQ-3><REQ-4><REQ-6>00000<REQ-1><REQ-2>',
  '<REQ-5>00000<REQ-4><REQ-2><REQ-6>00000<REQ-3><REQ-1>',
  '<REQ-5>00000<REQ-4><REQ-2><REQ-6>00000<REQ-1><REQ-3>',
  '<REQ-5>00000<REQ-4><REQ-3><REQ-6>00000<REQ-2><REQ-1>',
  '<REQ-5>00000<REQ-3><REQ-4><REQ-6>00000<REQ-2><REQ-1>',
  '<REQ-5>00000<REQ-2><REQ-3><REQ-6>00000<REQ-1><REQ-4>',
  '<REQ-5>00000<REQ-1><REQ-2><REQ-6>00000<REQ-4><REQ-3>',
  '<REQ-5>00000<REQ-1><REQ-4><REQ-6>00000<REQ-2><REQ

In [32]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["King", "Bay"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['1100001101000011',
 '1100001100100011',
 '1100001110000011',
 '0110001100100011',
 '1010001101000011',
 '1010001100100011',
 '0110001101000011',
 '0110001110000011',
 '1010001110000011']

In [33]:
fulFillment

{'1010001101000011': ['<REQ-7>0<REQ-5>000<REQ-2><REQ-3>0<REQ-6>0000<REQ-1><REQ-4>',
  '<REQ-5>0<REQ-7>000<REQ-2><REQ-4>0<REQ-6>0000<REQ-1><REQ-3>',
  '<REQ-7>0<REQ-5>000<REQ-3><REQ-2>0<REQ-6>0000<REQ-4><REQ-1>',
  '<REQ-5>0<REQ-7>000<REQ-3><REQ-2>0<REQ-6>0000<REQ-1><REQ-4>',
  '<REQ-5>0<REQ-7>000<REQ-4><REQ-1>0<REQ-6>0000<REQ-2><REQ-3>',
  '<REQ-7>0<REQ-5>000<REQ-2><REQ-4>0<REQ-6>0000<REQ-1><REQ-3>',
  '<REQ-7>0<REQ-5>000<REQ-1><REQ-2>0<REQ-6>0000<REQ-3><REQ-4>',
  '<REQ-7>0<REQ-5>000<REQ-1><REQ-3>0<REQ-6>0000<REQ-2><REQ-4>',
  '<REQ-7>0<REQ-5>000<REQ-1><REQ-3>0<REQ-6>0000<REQ-4><REQ-2>',
  '<REQ-5>0<REQ-7>000<REQ-2><REQ-3>0<REQ-6>0000<REQ-1><REQ-4>',
  '<REQ-5>0<REQ-7>000<REQ-4><REQ-2>0<REQ-6>0000<REQ-3><REQ-1>',
  '<REQ-5>0<REQ-7>000<REQ-1><REQ-3>0<REQ-6>0000<REQ-2><REQ-4>',
  '<REQ-5>0<REQ-7>000<REQ-4><REQ-3>0<REQ-6>0000<REQ-2><REQ-1>',
  '<REQ-5>0<REQ-7>000<REQ-2><REQ-1>0<REQ-6>0000<REQ-3><REQ-4>',
  '<REQ-7>0<REQ-5>000<REQ-1><REQ-2>0<REQ-6>0000<REQ-4><REQ-3>',
  '<REQ-5>0<REQ-7>00

In [34]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["King", "Ocean"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['1011001101000011',
 '1101001100100011',
 '0111001100100011',
 '1100011110000011',
 '1100101101000011',
 '0110011110000011',
 '0110011101000011',
 '0110101101000011',
 '1010101100100011',
 '1011001110000011',
 '1010101101000011',
 '0111001110000011',
 '1100011100100011',
 '1010011100100011',
 '1010101110000011',
 '1100101100100011',
 '1010011101000011',
 '1100101110000011',
 '1010011110000011',
 '0110011100100011',
 '1011001100100011',
 '1101001110000011',
 '1100011101000011',
 '0110101100100011',
 '1101001101000011',
 '0111001101000011',
 '0110101110000011']

In [35]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["King", "Ocean"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['1101101100100011',
 '0111011100100011',
 '1010111110000011',
 '0110111101000011',
 '0111011101000011',
 '1011101101000011',
 '1011011110000011',
 '1101101101000011',
 '0110111110000011',
 '1101011100100011',
 '1011101100100011',
 '0111101110000011',
 '0110111100100011',
 '1101101110000011',
 '1101011101000011',
 '1101011110000011',
 '1100111100100011',
 '0111011110000011',
 '0111101101000011',
 '1010111101000011',
 '1100111101000011',
 '1010111100100011',
 '1100111110000011',
 '1011011101000011',
 '1011011100100011',
 '0111101100100011',
 '1011101110000011']

In [36]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["King", "Ocean"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Congratulations Your Request Can be fulfilled


['0111111100100011',
 '1101111100100011',
 '1011111110000011',
 '1101111110000011',
 '0111111101000011',
 '1101111101000011',
 '0111111110000011',
 '1011111101000011',
 '1011111100100011']

In [37]:
promised, fulFillment = evaluateANewRequest(ProductsInAttributeSpace, promised, Attributes, request=["King", "Ocean"], 
                                           prevFulfillment = fulFillment)
promised[max(promised.keys())]

Sorry! Request can't be fulfilled


['0111111100100011',
 '1101111100100011',
 '1011111110000011',
 '1101111110000011',
 '0111111101000011',
 '1101111101000011',
 '0111111110000011',
 '1011111101000011',
 '1011111100100011']

In [38]:
fulFillment

{'1101111100100011': ['<REQ-5><REQ-7>0<REQ-8><REQ-10><REQ-9><REQ-3><REQ-4>00<REQ-6>000<REQ-2><REQ-1>',
  '<REQ-7><REQ-5>0<REQ-9><REQ-8><REQ-10><REQ-1><REQ-4>00<REQ-6>000<REQ-3><REQ-2>',
  '<REQ-7><REQ-5>0<REQ-8><REQ-10><REQ-9><REQ-4><REQ-3>00<REQ-6>000<REQ-1><REQ-2>',
  '<REQ-5><REQ-7>0<REQ-10><REQ-8><REQ-9><REQ-1><REQ-2>00<REQ-6>000<REQ-3><REQ-4>',
  '<REQ-7><REQ-5>0<REQ-10><REQ-9><REQ-8><REQ-4><REQ-3>00<REQ-6>000<REQ-2><REQ-1>',
  '<REQ-5><REQ-7>0<REQ-9><REQ-10><REQ-8><REQ-2><REQ-4>00<REQ-6>000<REQ-1><REQ-3>',
  '<REQ-5><REQ-7>0<REQ-8><REQ-10><REQ-9><REQ-4><REQ-2>00<REQ-6>000<REQ-1><REQ-3>',
  '<REQ-7><REQ-5>0<REQ-9><REQ-8><REQ-10><REQ-3><REQ-2>00<REQ-6>000<REQ-4><REQ-1>',
  '<REQ-7><REQ-5>0<REQ-10><REQ-9><REQ-8><REQ-4><REQ-1>00<REQ-6>000<REQ-2><REQ-3>',
  '<REQ-7><REQ-5>0<REQ-8><REQ-10><REQ-9><REQ-3><REQ-1>00<REQ-6>000<REQ-2><REQ-4>',
  '<REQ-7><REQ-5>0<REQ-9><REQ-10><REQ-8><REQ-1><REQ-2>00<REQ-6>000<REQ-3><REQ-4>',
  '<REQ-7><REQ-5>0<REQ-8><REQ-9><REQ-10><REQ-1><REQ-3>00<REQ-6>000<

In [23]:
fulFillment.index('0000000000000002')

2

In [30]:
1 << 100

1267650600228229401496703205376

In [31]:
testString = "<<<<<<<adsda><ADAD>a<REQ-1>"

In [32]:
find(testString, "<")

[0, 1, 2, 3, 4, 5, 6, 13, 20]

In [33]:
find(testString, ">")

[12, 18, 26]

In [85]:
import re

def encodeString(string):
    string = re.sub(r'<REQ-[0-9]+>', r'x', string)
    return string

def decodeString(stringAsList, encodedPortions, previousComposition, deltaLength):
    
    StEn = {}
    if deltaLength > 0:
        for i,j in encodedPortions.keys():
            ii = i + deltaLength
            jj = j + deltaLength
            StEn[(ii,jj)] = encodedPortions[(i,j)]
    else:
        StEn = encodedPortions
    
    i = 0
    j = 0
    for ii in range(len(stringAsList)):
        if stringAsList[ii] == 'x':
            stringAsList[ii] = previousComposition[i]
            i = i + 1
    
    decodedString =  ''.join(stringAsList)
    start = find(decodedString, ch = '<')
    end = find(decodedString, ch = '>')
    assert len(start) == len(end)
    if len(start)> 0:
        for i, j in zip(start, end):
            substr = decodedString[i:j+1]
            recordKeeping[(i,j)] = substr
    dif = set(StEn.items()) - set(recordKeeping.items())
    assert len(dif) == 0
    
    return decodedString
            
       
        

In [86]:
encodeString('1000<REQ-9>001000000<REQ-10>00000')

'1000x001000000x00000'

In [87]:
ex = '1000<REQ-9>001000000<REQ-10>00000'
start = find(ex, ch = '<')
end = find(ex, ch = '>')
assert len(start) == len(end)

In [88]:
previousComposition = []
recordKeeping = {}
if len(start)> 0:
    for i, j in zip(start, end):
        substr = ex[i:j+1]
        recordKeeping[(i,j)] = substr
        previousComposition.append(substr)

In [89]:
previousComposition

['<REQ-9>', '<REQ-10>']

In [90]:
recordKeeping

{(4, 10): '<REQ-9>', (20, 27): '<REQ-10>'}

In [91]:
encodedString = encodeString(ex)

In [92]:
encodedString

'1000x001000000x00000'

In [93]:
lb = len(encodedString)
encodedString = encodedString.rjust(100, '0')
la = len(encodedString)
delta = la - lb

encodedStringAsList = list(encodedString)
encodedStringAsList[50] = 'y'

In [94]:
decodedString = decodeString(encodedStringAsList, recordKeeping, previousComposition, deltaLength =delta)

In [84]:
del decodeString

In [95]:
decodedString

'00000000000000000000000000000000000000000000000000y000000000000000000000000000001000<REQ-9>001000000<REQ-10>00000'